# Data Preprocessing

## Library Import

In [10]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Import

### ISTAT

In [59]:
df_biblioteche = pd.read_csv('/content/drive/MyDrive/Dati KPMG/Biblioteche.csv', sep='|', encoding='latin-1')
print('[ISTAT] Numerosità file biblioteche: %d colonne - %d righe' % (df_biblioteche.shape[1], df_biblioteche.shape[0]))

df_internet = pd.read_csv('/content/drive/MyDrive/Dati KPMG/Internet.csv',sep="|")
print('[ISTAT] Numerosità file internet: %d colonne - %d righe' % (df_internet.shape[1], df_internet.shape[0]))

df_occupazione = pd.read_csv('/content/drive/MyDrive/Dati KPMG/Occupazione.csv', sep='|')
print('[ISTAT] Numerosità file occupazione: %d colonne - %d righe' % (df_occupazione.shape[1], df_occupazione.shape[0]))

df_inquinamento = pd.read_csv('/content/drive/MyDrive/Dati KPMG/inq_crim_rumore.csv')
print('[ISTAT] Numerosità file inquinamento criminalità e rumore: %d colonne - %d righe' % (df_inquinamento.shape[1], df_inquinamento.shape[0]))

df_famiglie =pd.read_csv('/content/drive/MyDrive/Dati KPMG/Aspetti della vita quotidiana - Famiglie - it.csv', sep='|')
print('[ISTAT] Numerosità file condizioni territorio: %d colonne - %d righe' % (df_famiglie.shape[1], df_famiglie.shape[0]))

df_stud_tot = pd.read_csv('/content/drive/MyDrive/Dati KPMG/laureati_tot_per_regione.csv', sep=',')
print('[ISTAT] Numerosità file laureati per regione: %d colonne - %d righe' % (df_stud_tot.shape[1], df_stud_tot.shape[0]))

[ISTAT] Numerosità file biblioteche: 17 colonne - 520 righe
[ISTAT] Numerosità file internet: 15 colonne - 39398 righe
[ISTAT] Numerosità file occupazione: 17 colonne - 52678 righe
[ISTAT] Numerosità file inquinamento criminalità e rumore: 11 colonne - 99 righe
[ISTAT] Numerosità file condizioni territorio: 13 colonne - 152952 righe
[ISTAT] Numerosità file laureati per regione: 17 colonne - 24398 righe


### MIUR

In [60]:
df_iscritti_residenza = pd.read_csv('/content/drive/MyDrive/Dati KPMG/14_iscrittixresidenzasedecorsogruppo.csv', encoding='latin-1', sep=';')
print('[MIUR] Numerosità file iscritti per residenza, corso, gruppo: %d colonne - %d righe' % (df_iscritti_residenza.shape[1], df_iscritti_residenza.shape[0]))

[MIUR] Numerosità file iscritti per residenza, corso, gruppo: 8 colonne - 534533 righe


## Processing

### ISTAT: Biblioteche

In [40]:
df_biblioteche_filtrato = df_biblioteche.drop(['ï»¿"ITTER107"','TIPO_DATO30', 'NATGIUR', 'TIPUTENZA', 'CITTADMAD', 'FIGPROFES', 'TIME', 'Flag Codes', 'Flags'], axis=1)
print('[ISTAT Debug] Colonne superflue eliminate')
df_biblioteche_filtrato = df_biblioteche_filtrato.loc[(df_biblioteche_filtrato['Tipo dato'] =='numero di posti per lettori') & 
                                                      (df_biblioteche_filtrato['Seleziona periodo'] ==2010) &
                                                      (df_biblioteche_filtrato['Territorio'] != 'Italia') &
                                                      (df_biblioteche_filtrato['Territorio'] != 'Nord-ovest') &
                                                      (df_biblioteche_filtrato['Territorio'] != 'Nord-est') &
                                                      (df_biblioteche_filtrato['Territorio'] != 'Centro') &
                                                      (df_biblioteche_filtrato['Territorio'] != 'Sud') &
                                                      (df_biblioteche_filtrato['Territorio'] != 'Isole')]

print('[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui')
df_biblioteche_filtrato = df_biblioteche_filtrato[['Territorio', 'Value']]
df_biblioteche_filtrato = df_biblioteche_filtrato.rename(columns={'Value': 'Biblioteche'})
print('[ISTAT Debug] Colonna valori rinominata')

[ISTAT Debug] Colonne superflue eliminate
[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui
[ISTAT Debug] Colonna valori rinominata


### ISTAT: Internet

In [41]:
df_internet_filtrato =df_internet.drop(['ITTER107','TIPO_DATO_SDS_ICT', 'SEXISTAT1', 'ETA1', 'TIP_FAM', 'TIME', 'Flag Codes', 'Flags'], axis=1)
print('[ISTAT Debug] Colonne superflue eliminate')
df_internet_filtrato = df_internet_filtrato.loc[~df_internet_filtrato['Territorio'].str.contains('0')]
print('[ISTAT Debug] Righe superflue eliminate')

df_internet_filtrato = df_internet_filtrato.loc[df_internet_filtrato['Tipo dato']=='connessione a banda larga non disponibile nella zona']
df_internet_filtrato = df_internet_filtrato.loc[df_internet_filtrato['Seleziona periodo']==2016]
df_internet_filtrato = df_internet_filtrato.loc[df_internet_filtrato['Tipologia familiare'].str.contains('totale')]
df_internet_filtrato = df_internet_filtrato.loc[(df_internet_filtrato['Territorio'] != 'Italia') &
                                                  (df_internet_filtrato['Territorio'] != 'Nord-ovest') &
                                                  (df_internet_filtrato['Territorio'] != 'Nord-est') &
                                                  (df_internet_filtrato['Territorio'] != 'Centro') &
                                                  (df_internet_filtrato['Territorio'] != 'Sud') &
                                                  (df_internet_filtrato['Territorio'] != 'Isole') &
                                                  ~(df_internet_filtrato['Territorio'].str.contains('metropolitana'))]

print('[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui')
df_internet_filtrato = df_internet_filtrato[['Territorio', 'Value']]

df_internet_filtrato['Territorio'].replace('Provincia Autonoma Bolzano / Bozen', 'Provincia autonoma di Bolzano', inplace=True)
df_internet_filtrato['Territorio'].replace('Provincia Autonoma Trento', 'Provincia autonoma di Trento', inplace=True)
df_internet_filtrato['Territorio'].replace('Friuli-Venezia Giulia', 'Friuli Venezia Giulia', inplace=True)
df_internet_filtrato['Territorio'].replace("Valle d'Aosta / Vallée d'Aoste","Valle D'Aosta", inplace=True)
df_internet_filtrato['Territorio'].replace('Emilia-Romagna', 'Emilia Romagna', inplace=True)
df_internet_filtrato['Territorio'].replace('Trentino Alto Adige / Südtirol', 'Trentino Alto Adige', inplace=True)
print('[ISTAT Debug] Correzione delle stringhe delle regioni effettuata')

df_internet_filtrato = df_internet_filtrato.rename(columns={'Value' : 'Internet'})
print('[ISTAT Debug] Colonna valori rinominata')

[ISTAT Debug] Colonne superflue eliminate
[ISTAT Debug] Righe superflue eliminate
[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui
[ISTAT Debug] Correzione delle stringhe delle regioni effettuata
[ISTAT Debug] Colonna valori rinominata


### ISTAT: Occupazione

In [42]:
df_occupazione_filtrata = df_occupazione.drop(['ITTER107','TIPO_DATO_FOL', 'SEXISTAT1', 'ETA1', 'TITOLO_STUDIO', 'CITTADINANZA', 'TIME','Flag Codes', 'Flags'], axis=1)
print('[ISTAT Debug] Colonne superflue eliminate')

df_occupazione_filtrata = df_occupazione_filtrata.loc[~df_occupazione_filtrata['Seleziona periodo'].str.contains('T')]
df_occupazione_filtrata = df_occupazione_filtrata.loc[df_occupazione_filtrata['Seleziona periodo'].str.contains('2021')]
df_occupazione_filtrata = df_occupazione_filtrata.loc[df_occupazione_filtrata['Sesso']=='totale']
df_occupazione_filtrata = df_occupazione_filtrata.loc[((df_occupazione_filtrata['Territorio'] != 'Italia') &
                                                  (df_occupazione_filtrata['Territorio'] != 'Nord-ovest') &
                                                  (df_occupazione_filtrata['Territorio'] != 'Nord-est') &
                                                  (df_occupazione_filtrata['Territorio'] != 'Centro') &
                                                  (df_occupazione_filtrata['Territorio'] != 'Sud') &
                                                  (df_occupazione_filtrata['Territorio'] != 'Isole'))]

df_occupazione_filtrata = df_occupazione_filtrata.loc[df_occupazione_filtrata['Titolo di studio']=='totale']
df_occupazione_filtrata = df_occupazione_filtrata.loc[df_occupazione_filtrata['Cittadinanza']=='totale']
df_occupazione_filtrata = df_occupazione_filtrata.loc[df_occupazione_filtrata['Classe di età']=='25-34 anni']
df_occupazione_filtrata = df_occupazione_filtrata.dropna(axis=1)
print('[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui')

df_occupazione_filtrata['Territorio'].replace('Provincia Autonoma Bolzano / Bozen', 'Provincia autonoma di Bolzano', inplace=True)
df_occupazione_filtrata['Territorio'].replace('Provincia Autonoma Trento', 'Provincia autonoma di Trento', inplace=True)
df_occupazione_filtrata['Territorio'].replace('Friuli-Venezia Giulia', 'Friuli Venezia Giulia', inplace=True)
df_occupazione_filtrata['Territorio'].replace("Valle d'Aosta / Vallée d'Aoste","Valle D'Aosta", inplace=True)
df_occupazione_filtrata['Territorio'].replace('Emilia-Romagna', 'Emilia Romagna', inplace=True)
df_occupazione_filtrata['Territorio'].replace('Trentino Alto Adige / Südtirol', 'Trentino Alto Adige', inplace=True)
print('[ISTAT Debug] Correzione delle stringhe delle regioni effettuata')

df_occupazione_filtrata = df_occupazione_filtrata[['Territorio', 'Value']]
df_occupazione_filtrata = df_occupazione_filtrata.rename(columns={'Value': 'Occupazione'})
print('[ISTAT Debug] Colonna valori rinominata')

[ISTAT Debug] Colonne superflue eliminate
[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui
[ISTAT Debug] Correzione delle stringhe delle regioni effettuata
[ISTAT Debug] Colonna valori rinominata


### ISTAT: Inquinamento, Criminalità e Rumore

In [43]:
df_inquinamento_fil = df_inquinamento[['Territorio', 'Presenza di problemi nella zona di residenza', 'Value']]
print('[ISTAT Debug] Colonne superflue eliminate')

df_inquinamento_fil = df_inquinamento_fil.loc[((df_inquinamento_fil['Territorio'] != 'Italia') &
                                                  (df_inquinamento_fil['Territorio'] != 'Nord-ovest') &
                                                  (df_inquinamento_fil['Territorio'] != 'Nord-est') &
                                                  (df_inquinamento_fil['Territorio'] != 'Centro') &
                                                  (df_inquinamento_fil['Territorio'] != 'Sud') &
                                                  (df_inquinamento_fil['Territorio'] != 'Isole') &
                                               ~(df_inquinamento_fil['Territorio'].str.contains('ab.')) & 
                                               ~(df_inquinamento_fil['Territorio'].str.contains('area')))]
print('[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui')

df_inquinamento_fil['Territorio'].replace('Provincia Autonoma Bolzano / Bozen', 'Provincia autonoma di Bolzano', inplace=True)
df_inquinamento_fil['Territorio'].replace('Provincia Autonoma Trento', 'Provincia autonoma di Trento', inplace=True)
df_inquinamento_fil['Territorio'].replace('Friuli-Venezia Giulia', 'Friuli Venezia Giulia', inplace=True)
df_inquinamento_fil['Territorio'].replace("Valle d'Aosta / Vallée d'Aoste","Valle D'Aosta", inplace=True)
df_inquinamento_fil['Territorio'].replace('Emilia-Romagna', 'Emilia Romagna', inplace=True)
df_inquinamento_fil['Territorio'].replace('Trentino Alto Adige / Südtirol', 'Trentino Alto Adige', inplace=True)
print('[ISTAT Debug] Correzione delle stringhe delle regioni effettuata')

df_trentino = df_inquinamento_fil.loc[(df_inquinamento_fil['Territorio']=='Trentino Alto Adige') | 
                                      (df_inquinamento_fil['Territorio']=='Provincia autonoma di Bolzano') | 
                                      (df_inquinamento_fil['Territorio']=='Provincia autonoma di Trento')]
df_trentino = df_trentino.set_index('Territorio').groupby(by=['Presenza di problemi nella zona di residenza']).mean()

df_trentino['Territorio'] = 'Trentino Alto Adige'
df_trentino = df_trentino.reset_index()
df_inquinamento_fil2 = df_inquinamento_fil.append(df_trentino)
print('[ISTAT Debug] Merge Trentino Alto Adige effettuato')

df_inquinamento_fil2 = df_inquinamento_fil2.rename(columns={'Presenza di problemi nella zona di residenza' : 'Problematica'})

df_inq = df_inquinamento_fil2.loc[df_inquinamento_fil2['Problematica']=='inquinamento']
df_crim = df_inquinamento_fil2.loc[df_inquinamento_fil2['Problematica']=='criminalità']
df_rum = df_inquinamento_fil2.loc[df_inquinamento_fil2['Problematica']=='rumori']

df_tot = df_inq.merge(df_crim, on='Territorio').merge(df_rum, on='Territorio').rename(columns={'Value_x':'Inquinamento', 'Value_y': 'Criminalita', 'Value':'Rumori'})
df_tot = df_tot[['Territorio', 'Inquinamento', 'Criminalita', 'Rumori']]
df_tot = df_tot.loc[(df_tot['Territorio']!='Provincia autonoma di Bolzano') & (df_tot['Territorio']!='Provincia autonoma di Trento')]
df_tot = df_tot.fillna(0)
print('[ISTAT Debug] Separazione dei valori in colonne effettuata')

df_tot = df_tot.append({'Territorio': 'Calabria', 'Inquinamento': 7.6, 'Criminalita':7.3, 'Rumori': 6.3}, ignore_index=True)
print('[ISTAT Debug] Inserimento valori di Calabria mancanti')

[ISTAT Debug] Colonne superflue eliminate
[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui
[ISTAT Debug] Correzione delle stringhe delle regioni effettuata
[ISTAT Debug] Merge Trentino Alto Adige effettuato
[ISTAT Debug] Separazione dei valori in colonne effettuata
[ISTAT Debug] Inserimento valori di Calabria mancanti


### ISTAT: Condizione del territorio

In [44]:
df_famiglie = df_famiglie.loc[((df_famiglie['TIPO_DATO_AVQ'] == 'HOUS_DIFFPARK_VQ') |
                                (df_famiglie['TIPO_DATO_AVQ'] == 'HOUS_DIRSTR_VQ') |
                                (df_famiglie['TIPO_DATO_AVQ'] == 'HOUS_TRAFFIC_VQ') |
                                (df_famiglie['TIPO_DATO_AVQ'] == 'HOUS_STRLIGHT_VQ'))]

df_famiglie = df_famiglie.loc[(df_famiglie['Misura']!='valori in migliaia')]
df_famiglie = df_famiglie.loc[(df_famiglie['TIME']==2017)]
df_famiglie = df_famiglie[['Territorio', 'TIPO_DATO_AVQ', 'Value']]
print('[ISTAT Debug] Colonne superflue eliminate')
print('[ISTAT Debug] Righe superflue eliminate')

df_famiglie = df_famiglie.loc[((df_famiglie['Territorio'] != 'Italia') &
                              (df_famiglie['Territorio'] != 'Nord-ovest') &
                              (df_famiglie['Territorio'] != 'Nord-est') &
                              (df_famiglie['Territorio'] != 'Centro') &
                              (df_famiglie['Territorio'] != 'Sud') &
                              (df_famiglie['Territorio'] != 'Isole') &
                              (df_famiglie['Territorio'] != 'Nord')&
                              (df_famiglie['Territorio'] != 'Mezzogiorno')&
                              ~(df_famiglie['Territorio'].str.contains('ab.')) & 
                              ~(df_famiglie['Territorio'].str.contains('area')))]
print('[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui')

df_famiglie['TIPO_DATO_AVQ'] = df_famiglie['TIPO_DATO_AVQ'].replace('HOUS_DIFFPARK_VQ', 'Parcheggio')
df_famiglie['TIPO_DATO_AVQ'] = df_famiglie['TIPO_DATO_AVQ'].replace('HOUS_DIRSTR_VQ', 'Strade')
df_famiglie['TIPO_DATO_AVQ'] = df_famiglie['TIPO_DATO_AVQ'].replace('HOUS_TRAFFIC_VQ', 'Traffico')
df_famiglie['TIPO_DATO_AVQ'] = df_famiglie['TIPO_DATO_AVQ'].replace('HOUS_STRLIGHT_VQ', 'Illuminazione')
print('[ISTAT Debug] Decodifica dei codici con le relative stringhe')

df_trentino2 = df_famiglie.loc[ (df_famiglie['Territorio']=='Trentino Alto Adige') | 
                                (df_famiglie['Territorio']=='Provincia Autonoma Bolzano / Bozen') | 
                                (df_famiglie['Territorio']=='Provincia Autonoma Trento') |
                                (df_famiglie['Territorio']=='Trentino Alto Adige / Südtirol')]
df_trentino2 = df_trentino2.set_index('Territorio').groupby(by='TIPO_DATO_AVQ').mean()

df_trentino2['Territorio'] = 'Trentino Alto Adige'
df_trentino2 = df_trentino2.reset_index()
df_famiglie = df_famiglie.append(df_trentino2)
print('[ISTAT Debug] Merge Trentino Alto Adige effettuato')

df_famiglie['Territorio'].replace('Provincia Autonoma Bolzano / Bozen', 'Provincia autonoma di Bolzano', inplace=True)
df_famiglie['Territorio'].replace('Provincia Autonoma Trento', 'Provincia autonoma di Trento', inplace=True)
df_famiglie['Territorio'].replace('Friuli-Venezia Giulia', 'Friuli Venezia Giulia', inplace=True)
df_famiglie['Territorio'].replace("Valle d'Aosta / Vallée d'Aoste","Valle D'Aosta", inplace=True)
df_famiglie['Territorio'].replace('Emilia-Romagna', 'Emilia Romagna', inplace=True)
print('[ISTAT Debug] Correzione delle stringhe delle regioni effettuata')

df_famiglie = df_famiglie.rename(columns={'TIPO_DATO_AVQ' : 'Problematica'})

df_str = df_famiglie.loc[df_famiglie['Problematica']=='Strade']
df_park = df_famiglie.loc[df_famiglie['Problematica']=='Parcheggio']
df_traf = df_famiglie.loc[df_famiglie['Problematica']=='Traffico']
df_ill = df_famiglie.loc[df_famiglie['Problematica']=='Illuminazione']

df_tot2 = df_str.merge(df_park, on='Territorio').merge(df_traf, on='Territorio').\
                rename(columns={'Value_x':'Strade', 'Value_y': 'Parcheggio', 'Value':'Traffico'}).\
                merge(df_ill, on='Territorio').\
                rename(columns={'Value':'Illuminazione'})
df_tot2 = df_tot2.loc[(df_tot2['Territorio']!='Provincia autonoma di Bolzano') & 
                      (df_tot2['Territorio']!='Provincia autonoma di Trento') & 
                      (df_tot2['Territorio']!='Trentino Alto Adige / Südtirol')]
df_tot2 = df_tot2[['Territorio', 'Strade', 'Parcheggio', 'Traffico', 'Illuminazione']]
print('[ISTAT Debug] Separazione dei valori in colonne effettuata')

df_tot2 = df_tot2.append({'Territorio': 'Calabria', 'Strade': 39.6, 'Parcheggio':35.3, 'Traffico': 28.5, 'Illuminazione':38.7}, ignore_index=True)
print('[ISTAT Debug] Inserimento valori di Calabria mancanti')

[ISTAT Debug] Colonne superflue eliminate
[ISTAT Debug] Righe superflue eliminate
[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui
[ISTAT Debug] Decodifica dei codici con le relative stringhe
[ISTAT Debug] Merge Trentino Alto Adige effettuato
[ISTAT Debug] Correzione delle stringhe delle regioni effettuata
[ISTAT Debug] Separazione dei valori in colonne effettuata
[ISTAT Debug] Inserimento valori di Calabria mancanti


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Problematica_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


### ISTAT: Laureati per regione

In [45]:
col = ['SEXISTAT1',  'AREADIDATTICA', 'TIPO_DATO6', 'NATGIUR','TIPCORSOLAUREA', 'ITTER107', 'TIME', 'Tipologia di corso di laurea', 'Tipo dato']
df_stud_tot = df_stud_tot.dropna(axis=1)
df_stud_tot = df_stud_tot.drop(col, axis=1)
print('[ISTAT Debug] Colonne superflue eliminate')

df_stud_punt = df_stud_tot.loc[(
    (df_stud_tot['Regione della sede didattica'] != 'Italia') &
    (df_stud_tot['Regione della sede didattica'] != 'Nord-ovest') &
    (df_stud_tot['Regione della sede didattica'] != 'Nord-est') &
    (df_stud_tot['Regione della sede didattica'] != 'Centro') &
    (df_stud_tot['Regione della sede didattica'] != 'Sud') &
    (df_stud_tot['Regione della sede didattica'] != 'Isole'))]

df_stud_punt = df_stud_punt.loc[
    (~df_stud_punt['Gruppo di corsi di laurea'].str.contains('totale')) &
    (df_stud_punt['Tipo Ateneo'] != 'totale')]
df_stud_punt = df_stud_punt.rename(columns={'Regione della sede didattica':'Territorio', 'Seleziona periodo':'Anno', 'Value':'Laureati', 'Gruppo di corsi di laurea':'Corso'})
print('[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui')

df_stud_punt['Territorio'].replace('Provincia Autonoma Bolzano / Bozen', 'Provincia autonoma di Bolzano', inplace=True)
df_stud_punt['Territorio'].replace('Provincia Autonoma Trento', 'Provincia autonoma di Trento', inplace=True)
df_stud_punt['Territorio'].replace('Friuli-Venezia Giulia', 'Friuli Venezia Giulia', inplace=True)
df_stud_punt['Territorio'].replace("Valle d'Aosta / Vallée d'Aoste","Valle D'Aosta", inplace=True)
df_stud_punt['Territorio'].replace('Emilia-Romagna', 'Emilia Romagna', inplace=True)
df_stud_punt['Territorio'].replace('Trentino Alto Adige / Südtirol', 'Trentino Alto Adige', inplace=True)
print('[ISTAT Debug] Correzione delle stringhe delle regioni effettuata')

df_stud_punt[['Parola','Corso Laurea']] = df_stud_punt['Corso'].str.split(' ',1, expand=True)
df_stud_punt = df_stud_punt.drop(['Parola', 'Corso'],axis=1)
df_stud_punt['Anno'].apply(str)

df_stud_punt_noBol = df_stud_punt.loc[(df_stud_punt['Territorio']=='Trentino Alto Adige') | 
                                      (df_stud_punt['Territorio']=='Provincia autonoma di Bolzano') | 
                                      (df_stud_punt['Territorio']=='Provincia autonoma di Trento')]
df_stud_punt_noBol = df_stud_punt_noBol.set_index('Territorio')

df_stud_punt_noBol = df_stud_punt_noBol.groupby(by=['Territorio','Tipo Ateneo','Sesso', 'Anno', 'Corso Laurea']).sum()
df_stud_punt_noBol = df_stud_punt_noBol.unstack(1).unstack(1).unstack(1).unstack(1).sum()
df_stud_punt_noBol = df_stud_punt_noBol.to_frame()
df_stud_punt_noBol = df_stud_punt_noBol.fillna(0)
df_stud_punt_noBol = df_stud_punt_noBol.rename(columns={0:'Laureati'})
df_stud_punt_noBol = df_stud_punt_noBol.reset_index().rename(columns={'level_0':'Territorio'})
df_stud_punt_noBol = df_stud_punt_noBol.replace('Laureati', 'Trentino Alto Adige')

df_stud_punt = df_stud_punt.loc[
    (df_stud_punt['Territorio']!='Trentino Alto Adige') & 
    (df_stud_punt['Territorio']!='Provincia autonoma di Bolzano') & 
    (df_stud_punt['Territorio']!='Provincia autonoma di Trento')]
    
df_stud_punt = df_stud_punt.append(df_stud_punt_noBol)
print('[ISTAT Debug] Merge Trentino Alto Adige effettuato')

[ISTAT Debug] Colonne superflue eliminate
[ISTAT Debug] Eliminati gli aggregati delle regioni e i tipi di dato superflui
[ISTAT Debug] Correzione delle stringhe delle regioni effettuata
[ISTAT Debug] Merge Trentino Alto Adige effettuato


### MIUR: Iscritti per regione, ateneo e corso di studi

In [61]:
df_iscritti_residenza_fil = df_iscritti_residenza.drop(['AteneoCOD', 'SedeP', 'GruppoCODICE'], axis=1)
df_iscritti_residenza_fil = df_iscritti_residenza_fil.rename(columns={'ResidenzaR':'Territorio'})
print('[MIUR Debug] Colonne superflue eliminate')

df_iscritti_residenza_fil = df_iscritti_residenza_fil.loc[(df_iscritti_residenza_fil['Territorio']!='Regione non fornita')&
                                                          (df_iscritti_residenza_fil['Territorio']!='Regione estera')]
print('[MIUR Debug] Tipi di dato superflui eliminati')

df_iscritti_residenza_fil['Territorio'] = df_iscritti_residenza_fil['Territorio'].str.capitalize()
df_iscritti_residenza_fil['Territorio'] = df_iscritti_residenza_fil['Territorio'].replace("Valle d'aosta", "Valle D'Aosta")
df_iscritti_residenza_fil['Territorio'] = df_iscritti_residenza_fil['Territorio'].replace("Friuli venezia giulia", "Friuli Venezia Giulia")
df_iscritti_residenza_fil['Territorio'] = df_iscritti_residenza_fil['Territorio'].replace("Emilia romagna", "Emilia Romagna")
print('[MIUR Debug] Correzione delle stringhe delle regioni effettuata')

df_iscritti_noBol = df_iscritti_residenza_fil.loc[(df_iscritti_residenza_fil['Territorio']=='Trentino Alto Adige') | 
                                      (df_iscritti_residenza_fil['Territorio']=='Provincia autonoma di bolzano') | 
                                      (df_iscritti_residenza_fil['Territorio']=='Provincia autonoma di trento')]
df_iscritti_noBol = df_iscritti_noBol.set_index('Territorio')

df_iscritti_noBol = df_iscritti_noBol.groupby(by=['Territorio','AnnoA','AteneoNOME', 'ResidenzaP']).sum()
df_iscritti_noBol = df_iscritti_noBol.unstack(1).unstack(1).unstack(1).sum()
df_iscritti_noBol = df_iscritti_noBol.to_frame().fillna(0).rename(columns={0:'Isc'}).reset_index().rename(columns={'level_0':'Territorio'}).replace('Isc', 'Trentino Alto Adige')
df_iscritti_residenza_fil = df_iscritti_residenza_fil.append(df_iscritti_noBol)
print('[MIUR Debug] Merge Trentino Alto Adige effettuato')

df_iscritti_residenza_fil[['Anno','Anno1']] = df_iscritti_residenza_fil['AnnoA'].str.split('/', 1, expand=True)
df_iscritti_residenza_fil = df_iscritti_residenza_fil.drop(['AnnoA', 'Anno1'], axis=1)
print('[MIUR Debug] Anno solare creato')

[MIUR Debug] Colonne superflue eliminate
[MIUR Debug] Tipi di dato superflui eliminati
[MIUR Debug] Correzione delle stringhe delle regioni effettuata
[MIUR Debug] Merge Trentino Alto Adige effettuato
[MIUR Debug] Anno solare creato


## Preprocessed DataFrame visualization

### ISTAT

In [47]:
print('[ISTAT] Visualizzazione DataFrame sulla Biblioteche')
df_biblioteche_filtrato

[ISTAT] Visualizzazione DataFrame sulla Biblioteche


,Territorio,Biblioteche
127,Piemonte,420
145,Liguria,99
163,Lombardia,523
199,Veneto,328
217,Friuli-Venezia Giulia,222
235,Emilia-Romagna,362
271,Toscana,789
289,Marche,104
307,Lazio,1982
343,Campania,828


In [48]:
print('[ISTAT] Visualizzazione DataFrame su Internet')
df_internet_filtrato

[ISTAT] Visualizzazione DataFrame su Internet


,Territorio,Internet
21495,Piemonte,1.3
21599,Valle D'Aosta,2.5
21703,Liguria,2.7
21807,Lombardia,1.5
22015,Trentino Alto Adige,4.4
22119,Provincia autonoma di Bolzano,4.6
22223,Provincia autonoma di Trento,4.2
22327,Veneto,1.9
22431,Friuli Venezia Giulia,1.0
22535,Emilia Romagna,1.6


In [49]:
print("[ISTAT] Visualizzazione DataFrame sull'Occupazione")
df_occupazione_filtrata

[ISTAT] Visualizzazione DataFrame sull'Occupazione


,Territorio,Occupazione
2811,Nord,74.958039
2911,Piemonte,73.496319
2923,Torino,70.667701
2935,Vercelli,68.752785
2947,Biella,75.398721
...,...,...
25237,Napoli,38.445847
25245,Benevento,47.007085
25385,Cosenza,47.499387
25501,Oristano,53.317465


In [50]:
print('[ISTAT] Visualizzazione DataFrame su Inquinamento, Criminalità e Rumore')
df_tot

[ISTAT] Visualizzazione DataFrame su Inquinamento, Criminalità e Rumore


,Territorio,Inquinamento,Criminalita,Rumori
0,Piemonte,10.40,10.3,14.7
1,Valle D'Aosta,0.00,0.0,0.0
2,Liguria,15.20,9.2,15.6
3,Lombardia,15.00,10.5,14.1
4,Veneto,9.60,6.8,7.8
5,Friuli Venezia Giulia,8.20,5.4,11.3
6,Emilia Romagna,14.70,16.3,11.2
7,Toscana,8.50,7.0,10.5
8,Umbria,6.40,10.7,5.8
9,Marche,6.20,5.8,6.6


In [51]:
print('[ISTAT] Visualizzazione DataFrame sulla Condizione del territorio')
df_tot2

[ISTAT] Visualizzazione DataFrame sulla Condizione del territorio


,Territorio,Strade,Parcheggio,Traffico,Illuminazione
0,Piemonte,24.200000,33.9,34.400000,31.500000
1,Valle D'Aosta,13.500000,20.5,20.800000,13.300000
2,Liguria,40.200000,52.7,41.800000,33.900000
3,Lombardia,26.800000,36.8,35.900000,26.500000
4,Veneto,19.800000,25.7,36.800000,30.200000
5,Friuli Venezia Giulia,20.000000,26.2,31.000000,19.700000
6,Emilia Romagna,20.000000,27.4,35.400000,29.900000
7,Toscana,25.500000,34.3,34.200000,31.500000
8,Umbria,26.100000,24.4,30.200000,30.600000
9,Marche,20.700000,30.2,31.800000,34.000000


In [52]:
print('[ISTAT] Visualizzazione DataFrame sui Laureati per regione')
df_stud_punt

[ISTAT] Visualizzazione DataFrame sui Laureati per regione


,Territorio,Tipo Ateneo,Sesso,Anno,Laureati,Corso Laurea
120,Piemonte,pubblica,maschi,2007,370.0,scientifico
121,Piemonte,pubblica,maschi,2008,340.0,scientifico
122,Piemonte,pubblica,maschi,2009,357.0,scientifico
123,Piemonte,pubblica,maschi,2010,369.0,scientifico
124,Piemonte,pubblica,maschi,2011,360.0,scientifico
...,...,...,...,...,...,...
635,Trentino Alto Adige,pubblica,maschi,2016,56.0,linguistico
636,Trentino Alto Adige,pubblica,maschi,2016,102.0,medico
637,Trentino Alto Adige,pubblica,maschi,2016,372.0,politico-sociale
638,Trentino Alto Adige,pubblica,maschi,2016,174.0,psicologico


### MIUR

In [62]:
print('[MIUR] Visualizzazione DataFrame sugli Iscritti per regione, ateneo e corso di studi')
df_iscritti_residenza_fil

[MIUR] Visualizzazione DataFrame sugli Iscritti per regione, ateneo e corso di studi


,AteneoNOME,Territorio,ResidenzaP,Isc,Anno
0,Roma La Sapienza,Lazio,ROMA,11213.0,2019
1,Roma La Sapienza,Lazio,ROMA,11187.0,2018
2,Roma La Sapienza,Lazio,ROMA,11137.0,2020
3,Roma La Sapienza,Lazio,ROMA,11105.0,2017
4,Roma La Sapienza,Lazio,ROMA,11052.0,2016
...,...,...,...,...,...
2019,Venezia Cà Foscari,Trentino Alto Adige,TRENTO,175.0,2020
2020,Venezia Iuav,Trentino Alto Adige,BOLZANO,14.0,2020
2021,Venezia Iuav,Trentino Alto Adige,TRENTO,96.0,2020
2022,Verona,Trentino Alto Adige,BOLZANO,691.0,2020


# Data Merging

### Creation Scoring DataFrame

In [70]:
df_punteggi = pd.DataFrame()

df_punteggi = df_internet_filtrato.merge(df_biblioteche_filtrato, on='Territorio', how='left').\
                merge(df_occupazione_filtrata, on='Territorio', how='inner').\
                merge(df_tot, on='Territorio', how='outer').\
                merge(df_tot2, on='Territorio', how='outer')
print('[ISTAT MergePunteggi] Merge della zona relativa agli score effettuata')

df_punt_group = df_punteggi.loc[
  (df_punteggi['Territorio']=='Trentino Alto Adige') | 
  (df_punteggi['Territorio']=='Provincia autonoma di Bolzano') | 
  (df_punteggi['Territorio']=='Provincia autonoma di Trento')]
df_punt_group = df_punt_group.set_index('Territorio')

df_punt_group = df_punt_group.mean()
df_punt_group = df_punt_group.to_frame().\
  rename(columns={0:'Trentino Alto Adige'}).\
  transpose().reset_index().\
  rename(columns={'index':'Territorio'})
df_punteggi = df_punteggi.append(df_punt_group)
print('[ISTAT MergePunteggi] Eventuale ricorrezione del record Trentino Alto Adige')

df_punteggi = df_punteggi.drop([1,5,6,7], axis=0)
df_punteggi = df_punteggi.fillna(0)
print('[ISTAT MergePunteggi] Rimozione record ridondanti sulle province autonome')

df_punteggi['Punteggio'] = (-6*df_punteggi['Internet'] + 0.01*df_punteggi['Biblioteche'] + 2*df_punteggi['Occupazione'])
df_punteggi = df_punteggi.sort_values('Punteggio', ascending=False)
print('[ISTAT MergePunteggi] Calcolo empirico dei punteggi basati su alcuni parametri (KPMG)')

[ISTAT MergePunteggi] Merge della zona relativa agli score effettuata
[ISTAT MergePunteggi] Eventuale ricorrezione del record Trentino Alto Adige
[ISTAT MergePunteggi] Rimozione record ridondanti sulle province autonome
[ISTAT MergePunteggi] Calcolo empirico dei punteggi basati su alcuni parametri (KPMG)


### Collection Merging

In [75]:
dataset = pd.concat([df_iscritti_residenza_fil, df_punteggi, df_stud_punt], join='outer')
dataset = dataset.fillna(0)
print('[DATASET Debug] Merge di tutte le collection in un unico DataFrame (ISTAT + MIUR)')

[DATASET Debug] Merge di tutte le collection in un unico DataFrame (ISTAT + MIUR)


### Visualization and Export of Complete Dataset

In [76]:
print('[DATASET] Visualizzazione del dataset completo')
dataset

[DATASET] Visualizzazione del dataset completo


,AteneoNOME,Territorio,ResidenzaP,Isc,Anno,Internet,Biblioteche,Occupazione,Inquinamento,Criminalita,Rumori,Strade,Parcheggio,Traffico,Illuminazione,Punteggio,Tipo Ateneo,Sesso,Laureati,Corso Laurea
0,Roma La Sapienza,Lazio,ROMA,11213.0,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0
1,Roma La Sapienza,Lazio,ROMA,11187.0,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0
2,Roma La Sapienza,Lazio,ROMA,11137.0,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0
3,Roma La Sapienza,Lazio,ROMA,11105.0,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0
4,Roma La Sapienza,Lazio,ROMA,11052.0,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,0,Trentino Alto Adige,0,0.0,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pubblica,maschi,56.0,linguistico
636,0,Trentino Alto Adige,0,0.0,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pubblica,maschi,102.0,medico
637,0,Trentino Alto Adige,0,0.0,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pubblica,maschi,372.0,politico-sociale
638,0,Trentino Alto Adige,0,0.0,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pubblica,maschi,174.0,psicologico


In [74]:
dataset.to_csv('Database_completo.csv', index=False, header=True)
print('[DATASET] Salvataggio del dataset completo in un file di tipo csv completato')

[DATASET] Salvataggio del dataset completo in un file di tipo csv completato
